## Experiment exp027-2
xlm-roberta-large, Batch Size: 32, Learning Rate: 2e-5, Warmup Steps: 500

In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    BertForTokenClassification,
    AutoModelForTokenClassification
)
import torch
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [56]:
test_comments_spans = pd.read_csv("./submissions/task2-predicted.csv")

In [57]:
test_comments_spans

,document,comment_id,type,start,end
0,NDY-004,2,compliment,0,21
1,NDY-004,4,affection declaration,0,19
2,NDY-004,5,affection declaration,0,25
3,NDY-004,5,affection declaration,26,56
4,NDY-004,5,positive feedback,57,71
...,...,...,...,...,...
5498,NDY-203,526,affection declaration,0,17
5499,NDY-203,526,positive feedback,30,59
5500,NDY-203,526,positive feedback,64,104
5501,NDY-203,526,affection declaration,105,106


In [58]:
test_comments

,document,comment_id,comment,predicted_labels,predicted_probs,offset_mapping,text_tokens,predicted_spans
0,NDY-004,1,Lol i love lochis,"[0, 0, 0, 0, 0, 0, 0, 0]","[[0.99999654, 1.7456429e-07, 1.6115715e-07, 1....","[[0, 0], [0, 1], [1, 3], [4, 5], [6, 10], [11,...","[▁L, ol, ▁i, ▁love, ▁loc, his]",[]
1,NDY-004,2,ihr singt voll gut :),"[0, 2, 12, 12, 12, 12, 12, 0]","[[0.9999976, 1.1218729e-07, 1.239344e-07, 1.50...","[[0, 0], [0, 3], [4, 8], [8, 9], [10, 14], [15...","[▁ihr, ▁sing, t, ▁voll, ▁gut, ▁:)]","[{'type': 'compliment', 'start': 0, 'end': 21,..."
2,NDY-004,3,Junge fick dich,"[0, 0, 0, 0, 0, 0]","[[0.9999981, 5.8623616e-08, 1.05891374e-07, 1....","[[0, 0], [0, 4], [4, 5], [6, 10], [11, 15], [0...","[▁Jung, e, ▁fick, ▁dich]",[]
3,NDY-004,4,Ihr seit die besten,"[0, 3, 13, 13, 13, 0]","[[0.99999774, 1.6417343e-07, 1.384722e-07, 1.1...","[[0, 0], [0, 3], [4, 8], [9, 12], [13, 19], [0...","[▁Ihr, ▁seit, ▁die, ▁besten]","[{'type': 'affection declaration', 'start': 0,..."
4,NDY-004,5,ihr seit die ALLER besten ich finde euch soooo...,"[0, 3, 13, 13, 13, 13, 13, 3, 13, 13, 13, 13, ...","[[0.99999785, 1.2960982e-07, 1.4320104e-07, 1....","[[0, 0], [0, 3], [4, 8], [9, 12], [13, 17], [1...","[▁ihr, ▁seit, ▁die, ▁ALLE, R, ▁besten, ▁ich, ▁...","[{'type': 'affection declaration', 'start': 0,..."
...,...,...,...,...,...,...,...,...
9224,NDY-203,522,hihi kannst du mich grüßen 💕 👋 😍 Achso wusstes...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 11, 0, 11, 11, ...","[[0.99999774, 1.8107521e-07, 1.0220851e-07, 9....","[[0, 0], [0, 4], [5, 11], [12, 14], [15, 19], ...","[▁hihi, ▁kannst, ▁du, ▁mich, ▁gr, üß, en, ▁, 💕...","[{'type': 'positive feedback', 'start': 27, 'e..."
9225,NDY-203,523,#Glocke aktiviert 👑 Ich liebe deine Videos 💍 💎...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 11, 11, 11, 11,...","[[0.9999976, 1.1908668e-07, 8.492378e-08, 6.60...","[[0, 0], [0, 1], [1, 2], [2, 6], [6, 7], [8, 1...","[▁#, G, lock, e, ▁aktiv, iert, ▁, 👑, ▁Ich, ▁li...","[{'type': 'positive feedback', 'start': 20, 'e..."
9226,NDY-203,524,Bist die beste ❤ Bitte Grüße mich 💕 ❤ 😘 😍,"[0, 3, 13, 13, 13, 13, 0, 0, 0, 1, 1, 11, 11, ...","[[0.9999974, 2.1362885e-07, 1.2580301e-07, 9.5...","[[0, 0], [0, 3], [3, 4], [5, 8], [9, 14], [15,...","[▁Bis, t, ▁die, ▁beste, ▁❤, ▁Bitte, ▁Grüße, ▁m...","[{'type': 'affection declaration', 'start': 0,..."
9227,NDY-203,525,"Hi Bonny ❤️ War letztens auf'm Flughafen , und...","[0, 0, 0, 0, 1, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.99999523, 6.63842e-07, 2.0147786e-07, 1.16...","[[0, 0], [0, 2], [3, 6], [6, 8], [9, 10], [10,...","[▁Hi, ▁Bon, ny, ▁❤, ️, ▁War, ▁letzten, s, ▁auf...","[{'type': 'positive feedback', 'start': 9, 'en..."


In [60]:
test_comments['has_spans'] = test_comments.apply(lambda x: len(x['predicted_spans']) > 0, axis=1)

In [63]:
test_comments['flausch'] = test_comments['has_spans'].map({True: 'yes', False: 'no'})

In [66]:
test_comments[["document","comment_id","flausch"]].to_csv(f'./submissions/task1-predicted.csv', index=False)

In [68]:
!cp './submissions/task1-predicted.csv' './submissions/subtask1_submission2.csv'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [70]:
!head -n 10 './submissions/task1-predicted.csv'

document,comment_id,flausch
NDY-004,1,no
NDY-004,2,yes
NDY-004,3,no
NDY-004,4,yes
NDY-004,5,yes
NDY-004,6,yes
NDY-004,7,no
NDY-004,8,yes
NDY-004,9,no


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
